# Neural Net with Pytorch
---

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms


from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from torchvision import models

from torch.autograd import Variable
import torch.nn.functional as F
print("is cuda available :",torch.cuda.is_available())

#basic imports
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# models
from torchvision import models

import os
print(os.listdir("../input"))

# Any results you write to the

In [ ]:
class CustomedDataSet(torch.utils.data.Dataset):
    def __init__(self, train=True):
        self.train = train
        if self.train :
            trainX = pd.read_csv('../input/train.csv')
            trainY = trainX.label.as_matrix().tolist()
            trainX = np.apply_along_axis(self.preprocess , axis=1 ,\
            arr = trainX.drop('label',axis=1).as_matrix().reshape(trainX.shape[0], 1, 28, 28).astype(np.uint8))
            self.datalist = trainX
            self.labellist = trainY
        else:
            testX = pd.read_csv('../input/test.csv')
            testX = testX.as_matrix().reshape(testX.shape[0], 1, 28, 28)
            self.datalist = testX
            
    def __getitem__(self, index):
        if self.train:
            return torch.Tensor(self.datalist[index].astype(float)),self.labellist[index]
        else:
            return torch.Tensor(self.datalist[index].astype(float))
    
    def __len__(self):
        return self.datalist.shape[0]
    
    def preprocess(self,img):
        print(img.shape)
        blur = cv2.GaussianBlur(img,(3,3),0)
        ret3,th3 = cv2.threshold(blur,120,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
        return th3

In [ ]:
train_dataset = CustomedDataSet()

test_dataset = CustomedDataSet(train=False)


In [ ]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True,
                                           num_workers=4)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)
len_td = len(train_dataset)

# now the required data is in loader
del train_dataset, test_dataset

# self made model pytorch

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        
        self.layer1 = nn.Sequential(
            nn.Conv2d(1 ,8, kernel_size=3,padding=2),
            nn.BatchNorm2d(8),
            nn.ReLU(),
            nn.MaxPool2d(2))
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(8, 32, kernel_size=5,padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU())
        
        self.layer3 = nn.Sequential(
            nn.Conv2d(32, 16, kernel_size=5,padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2))
        
        self.fc1 = nn.Linear(784, 128)
        self.fc2 = nn.Linear(128, 32)
        self.fc3 = nn.Linear(32, 10)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0), -1)
#         print(len(out.view(out.size(0), -1)))
        out = self.fc1(out)
        out = self.fc2(out)
        out = self.fc3(out)
        return out

cnn = CNN()
cnn.cuda()

---
# Pretrained models
---

In [ ]:
model = models.resnet18(pretrained = True) # tochvision

# display properties
model.parameters

# Training Loop


In [ ]:
num_epochs = 10
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = Variable(images).cuda()
        labels = Variable(labels).cuda()
        
        # initiallzing gradient to 0
        optimizer.zero_grad()
        
        #predicting the batch
        outputs = cnn(images)
        
        # difference or loss from actual
        loss = criterion(outputs,labels)
        
         if (i + 1)% 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, (i + 1) * len(images), len(train_loader.dataset),
                100. * (i + 1) / len(train_loader), loss.data[0]))


        
        # calculating the gradients
        loss.backward()
        
        # updating the gradient
        optimizer.step()

# TEsting Model

In [ ]:
# staring 
cnn.eval()

ans = torch.cuda.LongTensor()

for images in test_loader:
    images = Variable(images).cuda()
    
    #predicting the image
    outputs = cnn(images)
    
    
    _,predicted = torch.max(outputs.data, 1)
    ans = torch.cat((ans,predicted),0)
    
    
def evaluate(data_loader):
    model.eval()
    loss = 0
    correct = 0
    
    for data, target in data_loader:
        data, target = Variable(data, volatile=True), Variable(target)
        if torch.cuda.is_available():
            data = data.cuda()
            target = target.cuda()
        
        output = model(data)
        
        loss += F.cross_entropy(output, target, size_average=False).data[0]

        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()
        
    loss /= len(data_loader.dataset)
        
    print('\nAverage loss: {:.4f}, Accuracy: {}/{} ({:.3f}%)\n'.format(
        loss, correct, len(data_loader.dataset),
        100. * correct / len(data_loader.dataset)))

In [ ]:


ans = ans.cpu().numpy() 

aa = pd.DataFrame(ans)
aa.columns = ['Label']
Id = range(1,aa.size+1)
aa.insert(0, 'ImageId', Id) 

aa.head()

aa.to_csv('submit_pytorch.csv',index = False)
